## Fine-tuning a ResNet50 model for Chest X-Ray Classification

In [1]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

import torchvision.models as models
import torch.nn as nn

import torch.optim as optim

In [2]:
# Define your transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)), # Resize images to fit the model
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), # mean and std are set to the standard of the training data for resnet50
])

# Load your dataset
train_dataset = datasets.ImageFolder(root='/rds/general/user/sz2823/home/ML_project/Chest_XRay_Classification/Dataset/images/train', transform=transform)
val_dataset = datasets.ImageFolder(root='/rds/general/user/sz2823/home/ML_project/Chest_XRay_Classification/Dataset/images/validation', transform=transform)

# Data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)

In [3]:
val_loader 

In [22]:
import torch

# Check if CUDA (GPU support) is available
is_cuda_available = torch.cuda.is_available()
print("Is CUDA (GPU) available:", is_cuda_available)

# If CUDA is available, print the GPU name(s)
if is_cuda_available:
    gpu_count = torch.cuda.device_count()
    print(f"Number of GPU(s) available: {gpu_count}")
    for i in range(gpu_count):
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
else:
    print("CUDA is not available. Using CPU.")

Is CUDA (GPU) available: True
Number of GPU(s) available: 1
GPU 0: Quadro RTX 6000


In [23]:
# Load a pre-trained ResNet model
model = models.resnet50(pretrained=True)

# Modify the classifier to fit your dataset
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, len(train_dataset.classes)) # Adjust this based on your number of classes

# Move the model to GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Device: {device}')

model = model.to(device)

print("Model device:", next(model.parameters()).device)

Device: cuda
Model device: cuda:0


In [10]:
criterion = nn.CrossEntropyLoss().cuda()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [ ]:
from tqdm import tqdm

# Number of epochs is set to 10, which means the entire dataset will pass through the network 10 times
num_epochs = 10

# Starting the training process

for epoch in range(num_epochs):
    print(f'Starting epoch {epoch+1}/{num_epochs}')  # Add this line to print the current epoch
    model.train()  # Setting the model to training mode;
    running_loss = 0.0 # Initializing the running loss which will accumulate losses over an epoch
    
    # Looping over the training data loader to fetch batches
    for batch_idx, (images, labels) in enumerate(tqdm(train_loader, desc=f'Epoch {epoch+1}')):
        
        images, labels = images.to(device), labels.to(device)
            
        optimizer.zero_grad()  # Clearing the gradients so they won't accumulate
        
        # Forward pass through the model to get predictions
        outputs = model(images)
        # Calculating the loss between predictions and true labels
        loss = criterion(outputs, labels)
        
        # Backpropagating the loss to calculate gradients for each parameter
        loss.backward()
        # Updating the parameters based on the current gradient
        optimizer.step()
        
        # Adding the batch loss to the total loss for the epoch
        running_loss += loss.item()
        
    # Computing the average loss for this epoch by dividing by the number of batches
    avg_loss = running_loss / len(train_loader)
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}')

Starting epoch 1/10


Epoch 1:  44%|████▍     | 1252/2853 [32:26<43:32,  1.63s/it]

In [ ]:
# Validation step starts here
    
model.eval()  # Setting the model to evaluation mode;
    
with torch.no_grad():  # Disabling gradient calculations; reduces memory usage and speeds up computation
        
    correct = 0  # Initializing the count for correct predictions
    total = 0  # Initializing the count for total predictions
        
    # Looping over the validation data loader to fetch batches
    for images, labels in val_loader:
            
        images, labels = images.to(device), labels.to(device)
                
        outputs = model(images)  # Forward pass to get predictions
        _, predicted = torch.max(outputs.data, 1)  # Finding the class with the highest score as the predicted class
        total += labels.size(0)  # Adding the number of labels in the batch to the total count
        correct += (predicted == labels).sum().item()  # Counting how many predictions were correct
        
        # Calculating the accuracy as the percentage of correct predictions
        val_accuracy = 100 * correct / total
        
    print(f'Accuracy on validation set: {val_accuracy:.2f}%')